In [ ]:
import astropy.coordinates as coord
from astropy.io import fits
import astropy.table as at
import astropy.units as u
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from scipy.stats import binned_statistic_2d
from gala.mpl_style import hesperia_r, turbo, turbo_r
import gala.dynamics as gd

from totoro.data import datasets
from totoro.config import galcen_frame

In [ ]:
zvz_bins1 = (np.arange(-60, 60+1e-3, 1.5),
             np.arange(-1.75, 1.75+1e-3, 0.05))
vrange1 = (0.9, 1.1)

zvz_bins2 = (np.arange(-100, 100+1e-3, 1.5*1.5),
             np.arange(-2, 2+1e-3, 0.05*1.5))
vrange2 = (0.5, 1.1)

In [ ]:
fig, axes = plt.subplots(1, len(datasets), 
                         figsize=(1 + 5 * len(datasets), 5.5))

for i, (k, d) in enumerate(datasets.items()):
    ax = axes[i]
    
    galcen = d.c.transform_to(galcen_frame)
    
    w = gd.PhaseSpacePosition(galcen.data)
    Lz = w.angular_momentum()[2]
    Lz = -Lz.to(229*u.km/u.s * 8.122*u.kpc)
    
    if len(d) > 50000:
        zvz_bins = zvz_bins1
        vrange = vrange1
    else:
        zvz_bins = zvz_bins2
        vrange = vrange2
    
    stat = binned_statistic_2d(
        galcen.v_z.to_value(u.km/u.s), 
        galcen.z.to_value(u.kpc),
        Lz,
        statistic='mean',
        bins=zvz_bins)
    ax.pcolormesh(stat.x_edge, stat.y_edge,
                  stat.statistic.T, 
                  vmin=vrange[0], vmax=vrange[1],
                  cmap=turbo, rasterized=True)

    ax.set_title(k)
    
    ax.set_xlim(zvz_bins[0].min(), zvz_bins[0].max())
    ax.set_ylim(zvz_bins[1].min(), zvz_bins[1].max())

axes[0].set_xlabel(f'$v_z$ [{u.km/u.s:latex_inline}]')
axes[0].set_ylabel(f'$z$ [{u.kpc:latex_inline}]')
    
fig.tight_layout()
fig.set_facecolor('w')

In [ ]:
fig, axes = plt.subplots(1, len(datasets), 
                         figsize=(1 + 5 * len(datasets), 5.5))

for i, (k, d) in enumerate(datasets.items()):
    ax = axes[i]
    
    galcen = d.c.transform_to(galcen_frame)
    
    if len(d) > 50000:
        zvz_bins = zvz_bins1
        vrange = vrange1
    else:
        zvz_bins = zvz_bins2
        vrange = vrange2
    
    vphi = -(galcen.cylindrical.differentials['s'].d_phi * galcen.cylindrical.rho).to(u.km/u.s, u.dimensionless_angles())
    stat = binned_statistic_2d(
        galcen.v_z.to_value(u.km/u.s), 
        galcen.z.to_value(u.kpc),
        vphi.to_value(u.km/u.s),
        statistic='mean',
        bins=zvz_bins)
    ax.pcolormesh(stat.x_edge, stat.y_edge,
                  stat.statistic.T, 
                  vmin=229-20, vmax=229+20,
                  cmap=turbo, rasterized=True)

    ax.set_title(k)
    
    ax.set_xlim(zvz_bins[0].min(), zvz_bins[0].max())
    ax.set_ylim(zvz_bins[1].min(), zvz_bins[1].max())

axes[0].set_xlabel(f'$v_z$ [{u.km/u.s:latex_inline}]')
axes[0].set_ylabel(f'$z$ [{u.kpc:latex_inline}]')
    
fig.tight_layout()
fig.set_facecolor('w')

In [ ]:
zvz_bins1 = (np.arange(-60, 60+1e-3, 2),
             np.arange(-1.75, 1.75+1e-3, 0.075))
vrange1 = (0.9, 1.1)

zvz_bins2 = (np.arange(-100, 100+1e-3, 2*1.5),
             np.arange(-2, 2+1e-3, 0.075*1.5))
vrange2 = (0.5, 1.1)

In [ ]:
fig, axes = plt.subplots(1, len(datasets), 
                         figsize=(1 + 5 * len(datasets), 5.5))

for i, (k, d) in enumerate(datasets.items()):
    ax = axes[i]
    
    galcen = d.c.transform_to(galcen_frame)
    
    w = gd.PhaseSpacePosition(galcen.data)
    Lz = w.angular_momentum()[2]
    Lz = -Lz.to(229*u.km/u.s * 8.122*u.kpc)
    
    if len(d) > 50000:
        zvz_bins = zvz_bins1
        vrange = vrange1
    else:
        zvz_bins = zvz_bins2
        vrange = vrange2
    
    feh = d.get_elem_ratio('MG_AL')
    mask = ((np.abs(Lz.value - 1.) < 0.1) & 
            (feh > -3) & np.isfinite(feh) &
            (feh != 0))
    feh = feh - np.mean(feh[mask])
    
    try:
        std = np.nanstd(feh[mask].filled(np.nan))
    except:
        std = np.nanstd(feh[mask])
    
    stat = binned_statistic_2d(
        galcen.v_z.to_value(u.km/u.s)[mask], 
        galcen.z.to_value(u.kpc)[mask],
        feh[mask],
        statistic='mean',
        bins=zvz_bins)
    ax.pcolormesh(stat.x_edge, stat.y_edge,
                  stat.statistic.T, 
                  vmin=-std, vmax=std,
                  cmap=hesperia_r, rasterized=True)

    ax.set_title(k)
    
    ax.set_xlim(zvz_bins[0].min(), zvz_bins[0].max())
    ax.set_ylim(zvz_bins[1].min(), zvz_bins[1].max())
    
    ax.axhline(0, ls='--', color='w')
    ax.axvline(0, ls='--', color='w')

axes[0].set_xlabel(f'$v_z$ [{u.km/u.s:latex_inline}]')
axes[0].set_ylabel(f'$z$ [{u.kpc:latex_inline}]')
    
fig.tight_layout()
fig.set_facecolor('w')